In [1]:
import pandas as pd
from datasets import Dataset
import sys
from tqdm import tqdm
sys.path.append('../src/')
from utils.data.prompt_utils import convert_prompstlye, tuluform, splitter
from utils.data.dataproc import indcomp
from utils.eval.rewards import unique_nns

/u/prasanns/miniconda3/envs/fixenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
rjson = pd.read_json("../outputs/results/dynarmlogs/math/mathbigdata1b_1bjustoffpolicyrmupper.jsonl", orient='records', lines=True)

In [2]:
mtmp = Dataset.load_from_disk("../../rlhf-length-biases//data/nouns/dponounsynth//")

In [8]:
mtmp = mtmp.filter(lambda ex: ex['magnitude']!=0)

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 230026.80 examples/s]


In [10]:
mtmp = mtmp.shuffle(seed=0)

In [12]:
heldm = mtmp.select(range(300))
newm = mtmp.select(range(300, len(mtmp)))
# newm.save_to_disk("../outputs/data/math/offp40knotie/")
# heldm.save_to_disk("../outputs/data/math/held/")

Saving the dataset (1/1 shards): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 40212.56 examples/s]


In [18]:
# mathinit = newm.select(range(1600)).save_to_disk("../outputs/data/math/mathwarm/")

Saving the dataset (1/1 shards): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1600/1600 [00:00<00:00, 125282.57 examples/s]


In [5]:
nountmp = Dataset.load_from_disk("../../rlhf-length-biases/data/nouns/dponounsynth/")

In [6]:
def fixlab(ex):
    newscos = unique_nns([ex['response_j'],ex['response_k']])
    if newscos[0]<newscos[1]:
        tmpr = ex['response_k']
        ex['response_k'] = ex['response_j']
        ex['response_j'] = tmpr
        ex['score_j'] = newscos[1]
        ex['score_k'] = newscos[0]
    else:
        ex['score_j'] = newscos[0]
        ex['score_k'] = newscos[1]
    ex['magnitude'] = abs(newscos[1]-newscos[0])
    return ex

In [7]:
unnoun = nountmp.map(fixlab, num_proc=5)

In [8]:
# unnoun.select(range(300, len(unnoun))).save_to_disk("../outputs/data/unique_nns/fullnpref")

Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49700/49700 [00:01<00:00, 29914.05 examples/s]


In [11]:
match = 0
for i in tqdm(range(len(nountmp))):
    unscos = unique_nns([nountmp['response_j'][i],nountmp['response_k'][i]])
    if (nountmp['score_j'][i]>nountmp['score_k'][i])==(unscos[0]>unscos[1]):
        match +=1
print(match/len(nountmp))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3300/3300 [00:25<00:00, 128.20it/s]

0.8903030303030303


In [29]:
unnoun = unnoun.filter(lambda ex:ex['magnitude']>0)

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3300/3300 [00:00<00:00, 13494.54 examples/s]


In [30]:
len(unnoun)

3035

In [31]:
unnoun.save_to_disk("../outputs/data/unique_nns/warmdata")

Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3035/3035 [00:00<00:00, 59043.47 examples/s]


In [8]:
unique_nns([nountmp[0]['response_j'],nountmp[0]['response_k'] ])

[13.0, 5.0]

In [7]:
def acat(inp):
    a, b = splitter(inp)
    return a+" "+b

In [15]:
rjson['texts'] = [[acat(s) for s in r] for r in rjson['texts']]

In [16]:
rjson

,texts,rewards,golds,call
0,[(((3 * 7) + 1) + 8) = (21 + 8) = 29 = 34 = 34...,"[1.109375, 1.4609375]","[-14.0, -18.0]","[1, 1]"
1,[(((3 - 7) * (9 + 2)) + 2) = ((-4 * (9 + 2)) +...,"[1.6328125, 1.6328125]","[0.0, 0.0]","[1, 1]"
2,[(((8 + 3) + 3) - 5) = (11 + 3) = 14 = 17 = 17...,"[1.3515625, 1.3125]","[-14.0, -14.0]","[1, 1]"
3,[(((4 + 8) * 8) * 2) = ((12 * 8) * 2) = (96 * ...,"[1.328125, 1.1015625]","[0.0, -6.0]","[1, 1]"
4,[(((4 - 1) * (6 + 2)) - ((5 - 3) * (5 + 9))) =...,"[1.0859375, 1.0234375]","[-21.0, -54.0]","[1, 1]"
...,...,...,...,...
63995,[(((5 + 9) - 2) * 2) = (14 - 2) = 12 = 10 + 2\...,"[1.453125, 0.2314453125]","[-22.0, -30.0]","[4000, 4000]"
63996,[(((7 - 4) - (4 * 9)) * (8 + 8)) = (3 - (4 * 9...,"[-1.0390625, 4.5625]","[-50.0, -47.0]","[4000, 4000]"
63997,[(((9 * 3) * 1) - 2) = (27 * 1) = 27.\n27\n(27...,"[-0.220703125, 1.03125]","[-34.0, -23.0]","[4000, 4000]"
63998,[(((4 + 7) + 6) * 6) = (11 + 6) = 17.\n13\nWha...,"[-3.5, 12.5625]","[-43.0, -3.0]","[4000, 4000]"


In [17]:
def mroll_split(inp):
    ind = inp.index("=")+2
    return inp[:ind], inp[ind:]

In [18]:
def rollout_to_prefs(indf, ismath=True):
    prefdata = []
    sfunct = mroll_split if ismath else splitter
    for ind, row in indf.iterrows():
        for i in range(0, len(row['texts']), 2):
            
            q1, a1 = sfunct(row['texts'][i])
            q2, a2 = sfunct(row['texts'][i+1])
            assert q1==q2
            astrs = [a1, a2]
            indj, indk = indcomp(row['golds'][i:i+2], 0)
            tmp = {
                'question': q1,
                'response_j': astrs[indj],
                'response_k': astrs[indk],
                'score_j': row['golds'][i+indj], 
                'score_k': row['golds'][i+indk],
                'magnitude': row['golds'][indj] - row['golds'][indk]
            }
            prefdata.append(tmp)
            
    return Dataset.from_list(prefdata)

pdata = rollout_to_prefs(rjson)

In [21]:
pdata = pdata.shuffle(seed=0)

In [22]:
smallpdata = pdata.select(range(50000))

In [23]:
# smallpdata.save_to_disk("../outputs/data/eurusrm/offpolicydata1k")
# smallpdata.save_to_disk("../outputs/data/math/offp50k")

Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 217838.17 examples/s]


In [25]:
# mprefs = Dataset.load_from_disk("../outputs/data/math/mathprefdata/")

In [26]:
# mprefs[0]

{'question': '(((3 * 4) - 2) - 1) = ',
 'response_j': '((12 - 2) - 1) = (13 - 1) = 8 = 9 = 8 = 9 = 9\n\n(13 - 1) = 8 = 7 = 7 = 7\n\n12 = 7 = 7\n\n12 = 7 = 7\n\n12 = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7\n\nM = 7',
 'response_k': '((12 - 2) - 1) = (9 - 1) = 8 = 7 = 7 = 7 = 7 = 7 = 7 = 7 = 7 = 7 = -7 = -7 = -7 = -7 = -7 = -7 = -63 = -63 = -63 = -63 = -63 = -63 = 539 = 493 = -549 = -5191 = -5491 = -5491 = -4971 = -4971 =',
 'score_j': 3.4722222222222223,
 'score_k': 3.194444444444444,
 'magnitude': 0.2777777777777781}